In [1]:
def floyd_warshall(graph):
    """
    Floyd-Warshall algorithm to compute the shortest path between all pairs of nodes.

    Parameters:
    graph (dict): Directed weighted graph represented as an adjacency list.
                  Each key is a node, and the value is a list of tuples (neighbor, weight).

    Returns:
    tuple: A tuple containing:
           - shortest_distances (list of lists): Matrix of shortest distances.
           - next_hop (list of lists): Matrix for reconstructing shortest paths.
           - node_labels (list): List of graph nodes.
    """
    # Extract nodes and assign indices for matrix representation
    node_labels = list(graph.keys())
    node_index_map = {node: idx for idx, node in enumerate(node_labels)}
    num_nodes = len(node_labels)

    # Initialize the distance and next-hop matrices
    shortest_distances = [[float('inf')] * num_nodes for _ in range(num_nodes)]
    next_hop = [[None] * num_nodes for _ in range(num_nodes)]

    # Set self-loop distances to 0
    for i in range(num_nodes):
        shortest_distances[i][i] = 0

    # Populate distances based on graph edges
    for source_node in graph:
        for neighbor_node, weight in graph[source_node]:
            source_idx, neighbor_idx = node_index_map[source_node], node_index_map[neighbor_node]
            shortest_distances[source_idx][neighbor_idx] = weight
            next_hop[source_idx][neighbor_idx] = neighbor_node

    # Apply the Floyd-Warshall algorithm
    for k in range(num_nodes):  # Intermediate node
        for i in range(num_nodes):  # Source node
            for j in range(num_nodes):  # Destination node
                if shortest_distances[i][k] + shortest_distances[k][j] < shortest_distances[i][j]:
                    shortest_distances[i][j] = shortest_distances[i][k] + shortest_distances[k][j]
                    next_hop[i][j] = next_hop[i][k]

    return shortest_distances, next_hop, node_labels

def print_matrix(matrix, node_labels):
    """
    Pretty-print a matrix with node labels.

    Parameters:
    matrix (list of lists): Matrix to print.
    node_labels (list): List of nodes corresponding to matrix indices.
    """
    print("      ", "   ".join(f"{node:<3}" for node in node_labels))
    for i, row in enumerate(matrix):
        print(f"{node_labels[i]:<5} ", "  ".join(f"{val if val != float('inf') else '∞':<5}" for val in row))

def reconstruct_path(source, destination, next_hop, node_labels):
    """
    Reconstructs the shortest path from source to destination.

    Parameters:
    source (str): Start node.
    destination (str): End node.
    next_hop (list of lists): Path reconstruction matrix.
    node_labels (list): List of node labels.

    Returns:
    list: The sequence of nodes forming the shortest path.
    """
    node_index_map = {node: idx for idx, node in enumerate(node_labels)}
    source_idx, destination_idx = node_index_map[source], node_index_map[destination]

    if next_hop[source_idx][destination_idx] is None:
        return None  # No path exists

    path = [source]
    while source != destination:
        source = next_hop[source_idx][destination_idx]
        path.append(source)
        source_idx = node_index_map[source]
    return path

# Example graph definition
graph = {
    'A': [('B', 3), ('C', 5)],
    'B': [('C', 2), ('D', 6)],
    'C': [('B', 1), ('D', 4), ('E', 6)],
    'D': [('E', 2)],
    'E': [('A', 3), ('D', 7)]
}

# Execute the Floyd-Warshall algorithm
shortest_distances, next_hop, node_labels = floyd_warshall(graph)

# Display the shortest distances matrix
print("Shortest Distances Matrix:")
print_matrix(shortest_distances, node_labels)

# Example: Reconstruct and print a specific path
source_node, destination_node = 'A', 'E'
path = reconstruct_path(source_node, destination_node, next_hop, node_labels)
if path:
    print(f"\nShortest path from {source_node} to {destination_node}: {' -> '.join(path)}")
else:
    print(f"\nNo path exists from {source_node} to {destination_node}.")


Shortest Distances Matrix:
       A     B     C     D     E  
A      0      3      5      9      11   
B      11     0      2      6      8    
C      9      1      0      4      6    
D      5      8      10     0      2    
E      3      6      8      7      0    

Shortest path from A to E: A -> C -> E
